In [31]:
%pip install pandas 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\josha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
import sqlite3
import pandas as pd

DB_NAME = 'furniture_production.db'
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

print(f"Подключение к базе данных '{DB_NAME}' установлено.")

Подключение к базе данных 'furniture_production.db' установлено.


In [33]:

table_names = ['ProductTypes', 'Materials', 'Workshops', 'Products', 'ProductWorkshops']

print("--- Проверка количества записей в таблицах ---")
for table in table_names:
    query = f"SELECT COUNT(*) FROM {table}"
    cursor.execute(query)
    count = cursor.fetchone()[0]
    print(f"Таблица '{table}': {count} записей")

--- Проверка количества записей в таблицах ---
Таблица 'ProductTypes': 6 записей
Таблица 'Materials': 5 записей
Таблица 'Workshops': 12 записей
Таблица 'Products': 20 записей
Таблица 'ProductWorkshops': 130 записей


In [34]:
print("--- Таблица: ProductTypes ---")

sql_pt = "SELECT * FROM ProductTypes;"
df_pt = pd.read_sql_query(sql_pt, conn)
print(df_pt)

--- Таблица: ProductTypes ---
  product_type_name type_coefficient
0          Гостиные              3,5
1          Прихожие              5,6
2     Мягкая мебель                3
3           Кровати              4,7
4             Шкафы              1,5
5            Комоды              2,3


In [35]:
print("--- Таблица: Materials ---")

sql_m = "SELECT * FROM Materials;"
df_m = pd.read_sql_query(sql_m, conn)
print(df_m)

--- Таблица: Materials ---
                     material_name loss_percentage
0  Мебельный щит из массива дерева           0,80%
1               Ламинированное ДСП           0,70%
2                           Фанера           0,55%
3                              МДФ           0,30%
4                             None            None


In [36]:
print("--- Таблица: Workshops ---")

sql_w = "SELECT * FROM Workshops;"
df_w = pd.read_sql_query(sql_w, conn)
print(df_w)

--- Таблица: Workshops ---
                                        workshop_name   workshop_type  \
0                                           Проектный  Проектирование   
1                                           Расчетный  Проектирование   
2                                             Раскроя       Обработка   
3                                           Обработки       Обработка   
4                                           Сушильный           Сушка   
5                                            Покраски       Обработка   
6                                           Столярный       Обработка   
7   Изготовления изделий из искусственного камня и...       Обработка   
8                          Изготовления мягкой мебели       Обработка   
9   Монтажа стеклянных, зеркальных вставок и други...          Сборка   
10                                             Сборки          Сборка   
11                                           Упаковки          Сборка   

    Количество человек 

In [37]:
print("--- Таблица: Products (Первые 5 строк) ---")

sql_p = """
SELECT 
    product_type_name, 
    product_name, 
    article, 
    "Минимальная стоимость для партнера" AS Cost, -- Используем найденное имя
    main_material_name
FROM 
    Products 
LIMIT 5;
"""
df_p = pd.read_sql_query(sql_p, conn)
print(df_p)

--- Таблица: Products (Первые 5 строк) ---
  product_type_name                               product_name  article  \
0          Гостиные  Комплект мебели для гостиной Ольха горная  1549922   
1          Гостиные           Стенка для гостиной Вишня темная  1018556   
2          Прихожие                      Прихожая Венге Винтаж  3028272   
3          Прихожие           Тумба с вешалкой Дуб натуральный  3029272   
4          Прихожие               Прихожая-комплект Дуб темный  3028248   

        Cost               main_material_name  
0  160507,00  Мебельный щит из массива дерева  
1  216907,00  Мебельный щит из массива дерева  
2   24970,00               Ламинированное ДСП  
3   18206,00               Ламинированное ДСП  
4  177509,00  Мебельный щит из массива дерева  


In [38]:
print("--- Таблица: ProductWorkshops (Первые 5 строк) ---")

sql_pw = "SELECT * FROM ProductWorkshops LIMIT 5;"
df_pw = pd.read_sql_query(sql_pw, conn)
print(df_pw)

--- Таблица: ProductWorkshops (Первые 5 строк) ---
                                        product_name  \
0  Кровать с подъемным механизмом с матрасом 1600...   
1                                      Тумба под ТВ    
2                       Диван-кровать угловой Книжка   
3                           Диван модульный Телескоп   
4                                 Диван-кровать Соло   

                                       workshop_name Время изготовления, ч  
0  Изготовления изделий из искусственного камня и...                   2,0  
1  Изготовления изделий из искусственного камня и...                   2,7  
2                         Изготовления мягкой мебели                   4,2  
3                         Изготовления мягкой мебели                   4,5  
4                         Изготовления мягкой мебели                   4,7  


In [39]:
print("--- Проверка структуры таблицы ProductWorkshops ---")
sql_pragma_pw = "PRAGMA table_info(ProductWorkshops);"

df_info_pw = pd.read_sql_query(sql_pragma_pw, conn)
print(df_info_pw)

--- Проверка структуры таблицы ProductWorkshops ---
   cid                   name  type  notnull dflt_value  pk
0    0           product_name  TEXT        0       None   0
1    1          workshop_name  TEXT        0       None   0
2    2  Время изготовления, ч  TEXT        0       None   0


In [40]:
print("--- Проверка ВЗАИМОСВЯЗИ (Products, Types, Materials) ---")

sql_join_query = """
SELECT 
    p.product_name AS Продукт,
    pt.product_type_name AS Тип_Продукции,
    pt.type_coefficient AS Коэффициент_Типа,
    
    -- Используем точное имя столбца, найденное в PRAGMA
    p."Минимальная стоимость для партнера" AS Стоимость_Партнера, 
    
    m.material_name AS Основной_Материал,
    m.loss_percentage AS Процент_Потерь
FROM 
    Products p
JOIN 
    ProductTypes pt ON p.product_type_name = pt.product_type_name
JOIN 
    Materials m ON p.main_material_name = m.material_name
LIMIT 10;
"""

df_check = pd.read_sql_query(sql_join_query, conn)
print(df_check)

--- Проверка ВЗАИМОСВЯЗИ (Products, Types, Materials) ---
                                             Продукт  Тип_Продукции  \
0          Комплект мебели для гостиной Ольха горная       Гостиные   
1                   Стенка для гостиной Вишня темная       Гостиные   
2                              Прихожая Венге Винтаж       Прихожие   
3                   Тумба с вешалкой Дуб натуральный       Прихожие   
4                       Прихожая-комплект Дуб темный       Прихожие   
5                       Диван-кровать угловой Книжка  Мягкая мебель   
6                           Диван модульный Телескоп  Мягкая мебель   
7                                 Диван-кровать Соло  Мягкая мебель   
8                             Детский диван Выкатной  Мягкая мебель   
9  Кровать с подъемным механизмом с матрасом 1600...        Кровати   

  Коэффициент_Типа Стоимость_Партнера                Основной_Материал  \
0              3,5          160507,00  Мебельный щит из массива дерева   
1           

In [41]:
print("--- ФИНАЛЬНАЯ ПРОВЕРКА (Используем АБСОЛЮТНО ВСЕ подтвержденные имена) ---")

sql_full_join_query_final_final = """
SELECT 
    p.product_name AS Продукт,
    
    -- Стоимость
    p."Минимальная стоимость для партнера" AS Стоимость_Партнера, 
    
    pt.product_type_name AS Тип_Продукции,
    pt.type_coefficient AS Коэффициент_Типа,
    m.loss_percentage AS Процент_Потерь_Материала,
    
    w.workshop_name AS Название_Цеха,
    w.workshop_type AS Тип_Цеха,
    
    -- !!! КРИТИЧЕСКИЙ СТОЛБЕЦ: Время изготовления !!!
    pw."Время изготовления, ч" AS Время_на_Цех_Часы
    
FROM 
    Products p
JOIN 
    ProductTypes pt ON p.product_type_name = pt.product_type_name
JOIN 
    Materials m ON p.main_material_name = m.material_name
JOIN 
    ProductWorkshops pw ON p.product_name = pw.product_name
JOIN
    Workshops w ON pw.workshop_name = w.workshop_name
LIMIT 10;
"""

try:
    df_guaranteed = pd.read_sql_query(sql_full_join_query_final_final, conn)
    print(df_guaranteed)
    print("\nВСЕ 5 ТАБЛИЦ УСПЕШНО СВЯЗАНЫ.")
    
except Exception as e:
    print(f"\nНЕРЕАЛЬНАЯ ОШИБКА: {e}")

--- ФИНАЛЬНАЯ ПРОВЕРКА (Используем АБСОЛЮТНО ВСЕ подтвержденные имена) ---
                                     Продукт Стоимость_Партнера Тип_Продукции  \
0  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
1  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
2  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
3  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
4  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
5  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
6  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
7  Комплект мебели для гостиной Ольха горная          160507,00      Гостиные   
8           Стенка для гостиной Вишня темная          216907,00      Гостиные   
9           Стенка для гостиной Вишня темная          216907,00      Гостиные   

  Коэффициент_Типа Процент_Потерь

In [42]:
conn.close()